# Obtain data from Wikipage - Part 1

In [36]:
import requests
import pandas as pd

# Load data from Wikipage

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
df_list = pd.read_html(html)

type(df_list)

df = df_list[0]

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [37]:
df.shape

(288, 3)

In [38]:
# Check how many Borough have not assigned: 77
df.groupby('Borough').count()

,Postcode,Neighbourhood
Borough,,
Central Toronto,17,17
Downtown Toronto,37,37
East Toronto,7,7
East York,6,6
Etobicoke,45,45
Mississauga,1,1
North York,38,38
Not assigned,77,77
Queen's Park,1,1


In [39]:
# Ignore the cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']
df.shape
df.reset_index(drop=True, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [40]:
# Looks good - 77 less rows :-)
#df.loc[df['Postcode'] == 'M4B']
df.shape

(211, 3)

In [41]:
# Prepare result dataframe df_result

df_result = df.copy()
df_result['Neighbourhood']=''

# Drop dublicates - only one row per Postcode
df_result.drop_duplicates(keep='first', inplace=True)

#df_result.loc[df_result['Postcode'] == 'M4B']

df_result.reset_index(drop=True, inplace=True)

df_result.head()
df_result.shape

(103, 3)

In [42]:
#df_result.at[0,'Borough']
#df_result.loc[df_result['Postcode'] == 'M4B']

In [43]:
# Update Neigbourhood colum of result dataframe df_result per postcode and get all Neigbourhoods in one string (value)

for index_r, row_r in df_result.iterrows():
        #print('S_r:',index_r, row_r.Postcode, row_r.Neighbourhood)
        value=''
        for index, row in df.iterrows():
            #print(row.Postcode, row.Neighbourhood)
            if row_r.Postcode==row.Postcode:
                if value=='':
                    value = str(row.Neighbourhood)
                else:
                    value = value +', ' + str(row.Neighbourhood)
                #print('chacka', value)
            #print('_:', row.Neighbourhood)   
        df_result.at[index_r,'Neighbourhood']=value
        #print('E_r:',index_r,df_result.at[index_r,'Neighbourhood'])
        

In [44]:
# Check the results
df_result.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [45]:
# Check the results
pd.options.display.max_colwidth = 200
df_result.loc[df_result['Postcode'] == 'M5A']

,Postcode,Borough,Neighbourhood
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [46]:
# Look on all Neigbourhoods with 'Not assigned'
df_result.loc[df_result['Neighbourhood'] == 'Not assigned']

,Postcode,Borough,Neighbourhood
4,M7A,Queen's Park,Not assigned


In [47]:
# Update Neighbourhood with Queen's Park for this row
df_result.at[4,'Neighbourhood']="Queen's Park"

In [48]:
# Check the result
df_result.loc[df_result['Neighbourhood'] == "Queen's Park"]

,Postcode,Borough,Neighbourhood
4,M7A,Queen's Park,Queen's Park


In [49]:
df_result.shape

(103, 3)

# Get the latitude and the longitude coordinates of each neighborhood - Part 2

In [50]:
# Load csv with Geocodes

df_geocodes = pd.read_csv('https://cocl.us/Geospatial_data')
df_geocodes.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [51]:

# Change Kewy of df_result & df_geocodes
df_result.rename(columns={"Postcode": "PostalCode"}, inplace=True)

df_geocodes.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

In [52]:
# Merge df_result with df_geocodes - This is the final result
 
df_final_result = pd.merge(df_result, df_geocodes, on='PostalCode')
df_final_result.head(15)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [53]:
# Check Results - looks good :-)
df_final_result.loc[df_result['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [54]:
df_final_result.shape

(103, 5)

# Explore and cluster the neighborhoods in Toronto - Part 3

In [21]:
!pip install folium
import folium

     |████████████████████████████████| 92kB 13.5MB/s eta 0:00:01


In [92]:
# Show the Neighbourhood of Borough Downtown Toronto on the map
df_final_result=df_final_result[df_final_result.Borough == 'Downtown Toronto']
print (df_final_result.shape)
df_final_result.head(20)

(18, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
36,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
42,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
48,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


In [58]:

# create map of  Toronto using latitude and longitude values
latitude= 43.654260
longitude= -79.360636

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)


# add markers to map
for lat, lng, borough, neighborhood in zip(df_final_result['Latitude'], df_final_result['Longitude'], df_final_result['Borough'], df_final_result['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [59]:
#Define Foursquare Credentials and Version
CLIENT_ID = '4UIOJQDCN5IWVX4VK4RWNSKTCYBO5LSJWJDPJAWHLECKGJK5' # your Foursquare ID
CLIENT_SECRET = '4RBRPJNGE5XGR1F2GYFWZ04BNZ2EDZU4MNQ44I33VC0FXKFS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4UIOJQDCN5IWVX4VK4RWNSKTCYBO5LSJWJDPJAWHLECKGJK5
CLIENT_SECRET:4RBRPJNGE5XGR1F2GYFWZ04BNZ2EDZU4MNQ44I33VC0FXKFS


In [62]:
# Explore Neighborhoods in Downtown Toronto
# Let's create a function to repeat the same process to all the neighborhoods inDowntown Toronto

LIMIT = 100 # limit of number of venues returned by Foursquare API

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [64]:
# Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues

toronto_downtown_venues = getNearbyVenues(names=df_final_result['Neighbourhood'],
                                   latitudes=df_final_result['Latitude'],
                                   longitudes=df_final_result['Longitude']
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian Place, Underground city
Church and Wellesley


In [65]:
# Let's check the size of the resulting dataframe

print(toronto_downtown_venues.shape)
toronto_downtown_venues.head()

(1299, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [91]:
# Let's check how many venues were returned for each neighborhood
toronto_downtown_venues.groupby('Neighborhood').count()

(1299, 7)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,58,58,58,58,58,58
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",45,45,45,45,45,45
Central Bay Street,88,88,88,88,88,88
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15
Church and Wellesley,86,86,86,86,86,86
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


In [80]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_downtown_venues['Venue Category'].unique())))

There are 205 uniques categories.


In [87]:
# Analyze Each Neighborhood

# one hot encoding
toronto_downtown_onehot = pd.get_dummies(toronto_downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_downtown_onehot['Neighborhood_'] = toronto_downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_downtown_onehot.columns[-1]] + list(toronto_downtown_onehot.columns[:-1])

toronto_downtown_onehot = toronto_downtown_onehot[fixed_columns]

print(toronto_downtown_onehot.shape)
toronto_downtown_onehot.head()


(1299, 206)


,Neighborhood_,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_downtown_onehot.groupby('Neighborhood_').mean().reset_index()
toronto_grouped

,Neighborhood_,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.000000,...,0.00,0.0,0.00,0.020000,0.000000,0.00,0.010000,0.000000,0.01,0.000000
1,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.0,0.00,0.017241,0.000000,0.00,0.000000,0.000000,0.00,0.000000
2,"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.00,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000
3,"Cabbagetown, St. James Town",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.022222,0.000000,...,0.00,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.011364,0.000000,...,0.00,0.0,0.00,0.011364,0.000000,0.00,0.011364,0.000000,0.00,0.011364
5,"Chinatown, Grange Park, Kensington Market",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.0,0.00,0.050000,0.000000,0.04,0.010000,0.000000,0.00,0.000000
6,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000
7,Church and Wellesley,0.011628,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.023256,0.000000,...,0.00,0.0,0.00,0.000000,0.011628,0.00,0.000000,0.011628,0.00,0.011628
8,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.000000,...,0.00,0.0,0.00,0.020000,0.000000,0.00,0.010000,0.000000,0.00,0.000000
9,"Design Exchange, Toronto Dominion Centre",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.020000,0.000000,...,0.00,0.0,0.01,0.010000,0.000000,0.00,0.010000,0.000000,0.00,0.000000


In [90]:
# manhattan_grouped.shape
toronto_grouped.shape

(18, 206)

In [93]:
# Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood_']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood_'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2      Thai Restaurant  0.04
3  American Restaurant  0.04
4                  Bar  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2  Farmers Market  0.03
3        Beer Bar  0.03
4      Steakhouse  0.03


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0     Airport Service  0.19
1      Airport Lounge  0.12
2    Airport Terminal  0.12
3  Airport Food Court  0.06
4        Airport Gate  0.06


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.07
1                Park  0.07
2  Italian Restaurant  0.04
3         Pizza Place  0.04
4                 Pub  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.15
1  Italian Restaurant  0.06
2                Café  

In [94]:
# Let's put that into a pandas dataframe

# First, let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [127]:
# Now let's create the new dataframe and display the top 7 venues for each neighborhood.

import numpy as np

num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood_']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(18)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,American Restaurant,Thai Restaurant,Hotel,Steakhouse
1,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Seafood Restaurant,Beer Bar,Farmers Market,Bakery
2,"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Sculpture Garden,Coffee Shop,Boutique
3,"Cabbagetown, St. James Town",Coffee Shop,Park,Pub,Italian Restaurant,Bakery,Pizza Place,Restaurant
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Ice Cream Shop,Fried Chicken Joint
5,"Chinatown, Grange Park, Kensington Market",Café,Chinese Restaurant,Vegetarian / Vegan Restaurant,Bar,Vietnamese Restaurant,Mexican Restaurant,Bakery
6,Christie,Café,Grocery Store,Park,Italian Restaurant,Coffee Shop,Convenience Store,Nightclub
7,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Burger Joint,Gay Bar,Gastropub
8,"Commerce Court, Victoria Hotel",Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gastropub,Deli / Bodega
9,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Italian Restaurant,Restaurant,Seafood Restaurant,Deli / Bodega


In [128]:
# Cluster Neighborhoods

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood_', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:18] 


array([0, 0, 3, 0, 0, 2, 4, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0], dtype=int32)

In [129]:
df_final_result.dtypes

PostalCode        object
Borough           object
Neighbourhood     object
Latitude         float64
Longitude        float64
dtype: object

In [130]:
neighborhoods_venues_sorted.dtypes

Neighborhood             object
1st Most Common Venue    object
2nd Most Common Venue    object
3rd Most Common Venue    object
4th Most Common Venue    object
5th Most Common Venue    object
6th Most Common Venue    object
7th Most Common Venue    object
dtype: object

In [131]:
neighborhoods_venues_sorted = neighborhoods_venues_sorted.rename(columns={'Neighborhood': 'Neighbourhood'})

In [152]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_final_result

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!


(18, 13)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Café,Restaurant,Pub,Mexican Restaurant
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Bakery,Italian Restaurant,Fast Food Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Clothing Store,Gastropub
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Steakhouse,Seafood Restaurant,Beer Bar,Farmers Market,Bakery
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Ice Cream Shop,Fried Chicken Joint


In [153]:
# create map

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [162]:
# Examine Clusters
# Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

toronto_merged.rename(columns={"Cluster Labels": "ClusterLabels"}, inplace=True)

# Cluster 1
toronto_merged[toronto_merged.ClusterLabels == 0]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Café,Restaurant,Pub,Mexican Restaurant
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Bakery,Italian Restaurant,Fast Food Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Clothing Store,Gastropub
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Steakhouse,Seafood Restaurant,Beer Bar,Farmers Market,Bakery
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Ice Cream Shop,Fried Chicken Joint
30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,0,Coffee Shop,Café,Bar,American Restaurant,Thai Restaurant,Hotel,Steakhouse
36,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,0,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Brewery,Scenic Lookout
42,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,0,Coffee Shop,Café,Hotel,Italian Restaurant,Restaurant,Seafood Restaurant,Deli / Bodega
48,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gastropub,Deli / Bodega
92,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,0,Coffee Shop,Restaurant,Café,Beer Bar,Seafood Restaurant,Cocktail Bar,Hotel


In [163]:
# Cluster 2
toronto_merged[toronto_merged.ClusterLabels == 1]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
91,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Building,Dim Sum Restaurant,Event Space,Ethiopian Restaurant


In [166]:
# Cluster 3
toronto_merged[toronto_merged.ClusterLabels == 2]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
80,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049,2,Café,Restaurant,Bookstore,Japanese Restaurant,Italian Restaurant,Bar,Bakery
84,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,2,Café,Chinese Restaurant,Vegetarian / Vegan Restaurant,Bar,Vietnamese Restaurant,Mexican Restaurant,Bakery


In [167]:
# Cluster 4
toronto_merged[toronto_merged.ClusterLabels == 3]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
87,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",43.628947,-79.39442,3,Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Sculpture Garden,Coffee Shop,Boutique


In [168]:
# Cluster 5
toronto_merged[toronto_merged.ClusterLabels == 4]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4,Café,Grocery Store,Park,Italian Restaurant,Coffee Shop,Convenience Store,Nightclub
